In [2]:
!pip install -U -q google-generativeai

In [4]:
import numpy as np
import pandas as pd
import google.generativeai as genai
from google.colab import userdata

In [5]:
GOOGLE_API_KEY=userdata.get('SECRET_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [6]:
# listando os modelos que trabalham com embeddings
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [7]:
# Exemplo de embedding
# Os embeddings representam objetos do mundo real, como palavras, imagens ou vídeos, em uma forma que os computadores podem processar.
# Os embeddings permitem pesquisas de similaridade e são fundamentais para a IA.

title = "A próxima geraçãode IA para desenvolvedores e Google Workspace"
simple_text = ('A próxima geraçãode IA para desenvolvedores e Google Workspace'
              '\n'
              'Artigo completo: \n'
              '\n'
              'Gemini API & Google AI Studio: Uma maneira acessível de explorar e criar protótipos com aplicações de IA geneativa')

# não vamos gerar um generate_content como na aula passada, e sim um embed_content
embeddings = genai.embed_content(model='models/embedding-001',
                                 content=simple_text,
                                 title=title,
                                 task_type='RETRIEVAL_DOCUMENT')

#
print(embeddings)


{'embedding': [0.05246651, -0.037160374, -0.020723857, 0.024448317, 0.062286947, -0.0088911755, -0.021487858, -0.022680633, 0.06796061, 0.06402678, 0.012150298, 0.007954303, -0.041308936, -0.01740316, 0.013617859, -0.034104977, 0.021430908, -0.010798436, -0.029524656, 0.00013978123, -0.0010323006, 0.011322236, -0.02854657, -0.061507255, -0.017929511, 0.021604259, 0.016292479, -0.040963683, -0.030783195, 0.021600168, -0.05382923, 0.048688352, -0.034948315, 0.008989003, -0.038912598, -0.049019877, -0.022292929, -0.044973053, -0.0030421875, -0.005044959, 0.00850585, -0.080954425, -0.005910666, 0.023170987, -0.006759374, -0.024058234, 0.04489959, 0.048544403, 0.021739352, -0.06189268, 0.029464198, 0.029930752, 0.06279784, -0.04168903, -0.0048372233, -0.005958282, 0.025700964, -0.029137576, 0.031249385, 0.0022111442, -0.0033668904, 0.017449813, -0.029848963, 0.055807967, 0.022335896, -0.06457468, -0.037509352, -0.0025576858, 0.009372786, 0.050504573, 0.010876673, 0.019893182, 0.056286152, -

# Criando um exemplo pratico com embedding

In [69]:
# Vamos criar uma lista de documentos que serão consultados
DOCUMENT1 ={
    "Titulo": "Pagamento com cartão de crédito",
    "Conteudo": "Cartão de crédito, quanto mais você utiliza, mais benefícios tem. Além disso, bons cartões não cobram anuidade ou mensalidade e ainda oferecem benefícios como aumento de limite e descontos em parceiros. Isso sem contar o fato de que você ganha mais tempo para pagar as suas compras – e o valor não é acrescido de juros por conta disso."
}

DOCUMENT2 = {
    "Titulo": "Pagamento com dinheiro",
    "Conteudo": "Utilizar dinheiro em espécie ajuda a manter os gastos sob controle. O motivo é simples: o consumidor observa as notas desaparecendo de sua carteira, passando para o poder de outra pessoa. Esse benefício não é encontrado nos pagamentos com cartão de crédito"
}

DOCUMENT3 ={
    "Titulo": "Pagamento com PIX",
    "Conteudo": "As principais vantagens do Pix são a rapidez nas transferências, a disponibilidade 24/7 e a eliminação de intermediários, o que reduz as taxas da transação. Também é possível realizar transações apenas com o smartphone, substituindo a necessidade de dinheiro físico ou cartões."
}

# cria uma lista com os as formas de pagamento
documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [70]:
documents

[{'Titulo': 'Pagamento com cartão de crédito',
  'Conteudo': 'Cartão de crédito, quanto mais você utiliza, mais benefícios tem. Além disso, bons cartões não cobram anuidade ou mensalidade e ainda oferecem benefícios como aumento de limite e descontos em parceiros. Isso sem contar o fato de que você ganha mais tempo para pagar as suas compras – e o valor não é acrescido de juros por conta disso.'},
 {'Titulo': 'Pagamento com dinheiro',
  'Conteudo': 'Utilizar dinheiro em espécie ajuda a manter os gastos sob controle. O motivo é simples: o consumidor observa as notas desaparecendo de sua carteira, passando para o poder de outra pessoa. Esse benefício não é encontrado nos pagamentos com cartão de crédito'},
 {'Titulo': 'Pagamento com PIX',
  'Conteudo': 'As principais vantagens do Pix são a rapidez nas transferências, a disponibilidade 24/7 e a eliminação de intermediários, o que reduz as taxas da transação. Também é possível realizar transações apenas com o smartphone, substituindo a nec

In [71]:
# cria um dataframe com os valores da lista documents
df = pd.DataFrame(documents)
df

,Titulo,Conteudo
0,Pagamento com cartão de crédito,"Cartão de crédito, quanto mais você utiliza, m..."
1,Pagamento com dinheiro,Utilizar dinheiro em espécie ajuda a manter os...
2,Pagamento com PIX,As principais vantagens do Pix são a rapidez n...


In [72]:
# precisamos agora gerar os embeddings de cada conteudo apresentado no dataframe

# definindo o modelo a ser usado
model = "models/embedding-001"

# criando uma função para gerar o embedding dos conteudos, retornando somente o campo "embedding" do resultado
def embed_fn(title, text):
  return genai.embed_content(model = model,
                             content=text,
                             title=title,
                             task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [73]:
# vamos aplicar a funcao em todas as linhas do nosso dataframe, criando a coluna Embedding e gerando o embedding para cada uma
df['Embeddings'] = df.apply(lambda linha: embed_fn(linha['Titulo'], linha['Conteudo']), axis=1)
df

,Titulo,Conteudo,Embeddings
0,Pagamento com cartão de crédito,"Cartão de crédito, quanto mais você utiliza, m...","[0.017153619, -0.06051853, -0.030119002, -0.00..."
1,Pagamento com dinheiro,Utilizar dinheiro em espécie ajuda a manter os...,"[-0.005402004, -0.041232295, -0.056180224, -0...."
2,Pagamento com PIX,As principais vantagens do Pix são a rapidez n...,"[0.07247284, -0.007965422, -0.05159554, 0.0099..."


In [74]:
len(df.iloc[0]['Embeddings'])

768

In [75]:
# criando uma funcao para gerar uma consulta e obter a resposta do modelo
def gerar_e_buscar_consulta(consulta, base, model):
  # precisamos também gerar o embedding da consulta para fazer a comparacao com nossos conteudos dos documentos
  embedding_da_consulta = genai.embed_content(model = model,
                                              content=consulta,
                                              task_type="RETRIEVAL_QUERY")['embedding']

  # criando um produto escalar (empilhar)
  produtos_escalares = np.dot(np.stack(df['Embeddings']),embedding_da_consulta)

  # dentro dos produtos escalares tem uma referencia a cada um dos textos no dataframe e o valor que representa a distância
  # a funcao argmax obtem o valor que mais se destaca e retorna o indice da linha
  indice = np.argmax(produtos_escalares)

  return df.iloc[indice]['Conteudo']



In [95]:
consulta = "Preciso comprar um produto, mas quero segurança na operação."
resposta = gerar_e_buscar_consulta(consulta, df, model)
resposta

'As principais vantagens do Pix são a rapidez nas transferências, a disponibilidade 24/7 e a eliminação de intermediários, o que reduz as taxas da transação. Também é possível realizar transações apenas com o smartphone, substituindo a necessidade de dinheiro físico ou cartões.'

In [96]:
# O tradoff da solução acima, e evita alucinações do modelo, mas ao mesmo tempo, o modelo perde a criatividade para gerar as respostas.
# RAG é o meio de combinar as coisas (busca na base de conhecimento do gemini e mescla com suas repostas de conteudo)

prompt = f'Reescreva de uma forma mais descontraída, sem adicionar informações que não façam parte do texto: {resposta}'

# configurando os parametros
generation_config = {
    "candidate_count":1, #modelo pode dar mais de uma resposta, vamos escolher a primeira resposta
    "temperature":1 # 0 -> modelo pouco criativo, 1-> modelo muito criativo

}

model2 = genai.GenerativeModel("gemini-1.0-pro", generation_config=generation_config)
response = model2.generate_content(prompt)
print(response.text)

O Pix é tipo aquele amigo que sempre tá por perto quando você precisa! Pode mandar e receber grana rapidinho, sem ter que ficar esperando aquela lentidão. E nem precisa se preocupar com horário, ele tá disponível 24/7, até quando o galo canta!

Sabe aqueles caras que ficam no meio, levando uma bufunfa pra fazer uma transferência? O Pix dá no pé deles! Isso significa que as taxas são menores, sobra mais grana no seu bolso.

E o melhor: dá pra fazer tudo isso só com o celular. Nada de carregar dinheiro ou correr atrás de cartão, o Pix resolve tudo rapidão!
